### Qgis, centerline add length, convert to point ,add index
```
Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'length', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 0, 'FORMULA' : '$length', 'INPUT' : 'G:/810_change_field/linestrings_single_clip.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp' }

Algorithm 'Extract vertices' starting…
Input parameters:
{ 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp' }

Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'index', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 1, 'FORMULA' : '@row_number', 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice_index.shp' }

```


### Qgis,centerline get width, export to csv

```
Algorithm 'Distance to nearest hub (line to hub)' starting…
Input parameters:
{ 'FIELD' : 'id', 'HUBS' : 'G:/811/batiment_single_polygon_clip_index_boundary_densify3_point.shp', 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice_index.shp', 'OUTPUT' : 'G:/811/centerline_width.shp', 'UNIT' : 0 }
```

### etree csv to xml

In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
root = ET.Element ('network')
root.set('version','1.0')
nodes=ET.SubElement(root, 'nodes')
links=ET.SubElement(root, 'links')

In [3]:
dict_hub = {}

In [4]:
with open ("centerline_width.csv",'rt',encoding='utf-8') as f:
    current_group = None
    current_hub=None
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT, FID,length, vertex_ind, vertex_par, vertex_p_1, distance, angle, index,HubName,HubDist = row
        
        #single_node
        ET.SubElement(nodes, 'node',{'nodeId': index, 'geometry':WKT[19:-2].split(',')[0], 'linkId':FID, 'vertex_ind':vertex_ind})
        #links
        ##id length
        if current_group is None or FID != current_group.get('linkId'):
            current_group = ET.SubElement(links, 'link',{'linkId':FID,'length':length })
        
        ## nodes
        if (vertex_ind=='0'):
            attributes = ET.SubElement(current_group,'attributes')
            attribute = ET.SubElement(attributes, 'attribute',{'Type': 'Topology','name':'nodes','nodeId':index})   
        else: 
            list_nodeId = attribute.get('nodeId')
            attribute.set('nodeId',list_nodeId+" "+index)
        ## width
        if FID not in dict_hub.keys():
            dict_hubDist = {}
            dict_hubDist [HubName] = HubDist
            dict_hub[FID]= dict_hubDist
        else:
            if (HubName in dict_hub[FID].keys()):
                dict_hub[FID][HubName]= dict_hub[FID][HubName] + " "+ HubDist
            else:
                dict_hub[FID][HubName]= HubDist
            
        

In [5]:
##set node1 node2 
for link in links.findall('link'):
    for attributes in link.findall('attributes'):
        for attribute in attributes.findall('attribute'):
            if (attribute.get('name')=='nodes'):
                list_node = attribute.get('nodeId').split(' ')
                link.set('node1', list_node[0])
                link.set('node2',list_node[-1])

In [6]:
##set list width
for link in links.findall('link'):
    key = link.get('linkId')
    list_width = ""
    for k, v  in dict_hub[key].items():
        avg=0
        list_str = v.split(' ')
        for w in list_str:
            avg= avg+float(w)
        avg = avg/ len(list_str)  
        list_width = list_width +" "+ str(avg)
    for attributes in link.findall('attributes'):
        ET.SubElement(attributes, "attribute",{'Type':'Topology','name':'width','width':list_width})

In [7]:
#add osm info to centerline

In [8]:
osm_info_tree = ET.parse('osm.xml')
osm_info_root = osm_info_tree.getroot()

In [9]:
way_to_link={}
osm_dict ={}
for osm_links in osm_info_root.findall('links'):
    for osm_lk in osm_links.findall('link'):
        
        #attrib
        if (osm_lk.get('edge_centrality') ==None):
            list_attr= [osm_lk.get('freespeed'),osm_lk.get('capacity'),osm_lk.get('permlanes'),
                    osm_lk.get('oneway'),osm_lk.get('modes')," "]
        else:
            list_attr= [osm_lk.get('freespeed'),osm_lk.get('capacity'),osm_lk.get('permlanes'),
                    osm_lk.get('oneway'),osm_lk.get('modes'),osm_lk.get('edge_centrality')]
        
        osm_dict[osm_lk.get('id')] = list_attr
        #way id
        for osm_attrs in osm_lk.findall('attributes'):
            for osm_attr in osm_attrs.findall('attribute'):
                if (osm_attr.get('name')=="osm:way:id"):
                    if osm_attr.text not in way_to_link.keys():
                        way_to_link[osm_attr.text] = [osm_lk.get('id')]
                    else:
                        if osm_lk.get('id') not in way_to_link[osm_attr.text]:
                            way_to_link[osm_attr.text].append(osm_lk.get('id'))

In [10]:
#print(way_to_link)
print (osm_dict['491067'], osm_dict['491068'], osm_dict['491069'], osm_dict['491070'])
#print (osm_dict)

['4.166666666666667', '480.0', '1.0', '1', 'car,car_passenger', ' '] ['4.166666666666667', '480.0', '1.0', '1', 'car,car_passenger', ' '] ['4.166666666666667', '480.0', '1.0', '1', 'car,car_passenger', ' '] ['4.166666666666667', '480.0', '1.0', '1', 'car,car_passenger', ' ']


In [11]:
## set osm 
osm_tree = ET.parse('centerline_osmid.xml')
osm_root = osm_tree.getroot()
centerline_to_osm ={}
for link in links.findall('link'):
    #add sub element list_osm
    for centerline in osm_root.findall('centerline'):
        if (centerline.get('centerline_id')== link.get('linkId')):
            #osmid
            for attributes in link.findall('attributes'):
                ET.SubElement(attributes, "attribute",{'Type':'Topology','name':'osm_way','way_id':centerline.get('osm_id')})
                
                for way in centerline.get('osm_id').split(' '):
                    if way in way_to_link.keys():
                        centerline_to_osm[link.get('linkId')]=way_to_link[way]

In [12]:
print (centerline_to_osm['26221'])

['491067', '491068', '491069', '491070']


In [13]:
## add 'freespeed','capacity','permlanes','oneway','modes'
for link in links.findall('link'):
    if link.get('linkId') in centerline_to_osm.keys():
        for attributes in link.findall('attributes'):
            
            ET.SubElement(attributes, 'attribute',{'Type':'Topology', 'name':'osm_link','link_id': ' '.join(centerline_to_osm[link.get('linkId')])})
            
            parametre = ['freespeed','capacity','permlanes','oneway','modes','edge_centrality']
            for par in parametre:
                index = parametre.index(par)
                for value in centerline_to_osm[link.get('linkId') ]:

                    b = False
                    a = None 
                    for attribute in attributes.findall('attribute'):
                        attr_name = attribute.get('name')
                        if (attr_name == par):
                            b=True
                            a=attribute
                            
                    if (b):
                        if (a.get(par) != None):
                            if osm_dict[value][index] !=None:
                                a.set(par, a.get(par)+' '+ osm_dict[value][index])
                            else:
                                a.set(par, a.get(par)+' ')
                        else:
                            if osm_dict[value][index] !=None:
                                a.set(par, ' '.join(osm_dict[value])+' ')
                            else:
                                a.set(par, ' ')
                                
                    else:
                        a= ET.SubElement(attributes, 'attribute',{'Type':'Topology', 'name':par,par:osm_dict[value][index]})

             

### add modes

```
processing.run("native:lineintersections", { 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/bus_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/bus_intersection.shp' })

processing.run("native:lineintersections", { 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/subway_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/subway_intersection.shp' })

processing.run("native:lineintersections",{ 'INPUT' : 'G:/809_add_subway_train/linestrings_buildings.shp', 'INPUT_FIELDS' : ['FID','vertex_ind'], 'INTERSECT' : 'G:/809_add_subway_train/train_line.shp', 'INTERSECT_FIELDS' : ['relation_i','relation_n','osm_id'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/807_centerline_xml/train_intersection.shp' })

```

### convert shp to csv to xml

In [14]:
import csv

In [15]:
dict_bus ={}
with open ("bus_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_bus.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_bus[FID]= dict_osm
        else:
            if osm_id in dict_bus[FID].keys():
                if {relation_i:relation_n} not in dict_bus[FID][osm_id] :
                    dict_bus[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_bus[FID]= dict_osm

In [16]:
for key,value in dict_bus.items():
    print(key,value)
    break

14 {'283473009': [{'1253894': 'Bus 72 : Gare de Lyon → Parc de Saint-Cloud'}]}


In [17]:
dict_subway ={}
with open ("subway_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_subway.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_subway[FID]= dict_osm
        else:
            if osm_id in dict_subway[FID].keys():
                if {relation_i:relation_n} not in dict_subway[FID][osm_id] :
                    dict_subway[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_subway[FID]= dict_osm    

In [18]:
for key,value in dict_subway.items():
    print(key,value)
    break

15 {'24380725': [{'123768': "Métro 10: Boulogne - Pont de Saint-Cloud → Gare d'Austerlitz"}, {'3328740': "Métro 10: Gare d'Austerlitz → Boulogne - Pont de Saint-Cloud"}]}


In [19]:
dict_train ={}
with open ("train_intersection.csv",'rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT,FID,vertex_ind,relation_i,relation_n,osm_id = row
        if FID not in dict_train.keys():
            dict_osm = {}
            dict_osm [osm_id] = [{relation_i:relation_n}]
            dict_train[FID]= dict_osm
        else:
            if osm_id in dict_train[FID].keys():
                if {relation_i:relation_n} not in dict_train[FID][osm_id] :
                    dict_train[FID][osm_id].append({relation_i:relation_n})
            else:
                dict_osm = {}
                dict_osm [osm_id] = [{relation_i:relation_n}]
                dict_train[FID]= dict_osm

In [20]:
for key,value in dict_train.items():
    print(key,value)
    break

711 {'181567868': [{'3047780': 'RER C Tronçon C7 - Champ de Mars - Tour Eiffel → Saint-Quentin-en-Yvelines'}, {'3049185': 'RER C Tronçon C5 - Champ de Mars - Tour Eiffel → Versailles Rive Gauche'}]}


In [22]:
for link in links.findall('link'):
    linkid = link.get('linkId')
    
    if linkid in dict_bus.keys():
        
        for attributes in link.findall('attributes'):
            b=False
            attr_bus=None
            mode ='bus'

            for way_id , ligne in dict_bus[linkid].items():
                for lg in ligne:
                    for relation_i, relation_n in lg.items():
                        
                        for attribute in attributes.findall('attribute'):
                            if (attribute.get('name')==mode):
                                b=True
                                attr_bus=attribute
                                break
                        
                        ligne_name=relation_n.split(' ')
                        if (b):
                            attr_bus.set(mode, attr_bus.get(mode)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                        else:
                            ET.SubElement(attributes, 'attribute',{'Type':'Transport','name':mode,mode: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})

In [23]:
for link in links.findall('link'):
    linkid = link.get('linkId')
    
    if linkid in dict_subway.keys():
        
        for attributes in link.findall('attributes'):
            b=False
            attr_subway=None
            mode ='subway'

            for way_id , ligne in dict_subway[linkid].items():
                for lg in ligne:
                    for relation_i, relation_n in lg.items():
                        
                        for attribute in attributes.findall('attribute'):
                            if (attribute.get('name')==mode):
                                b=True
                                attr_subway=attribute
                                break
                        
                        ligne_name=relation_n.split(' ')
                        if (b):
                            attr_subway.set(mode, attr_subway.get(mode)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                        else:
                            ET.SubElement(attributes, 'attribute',{'Type':'Transport','name':mode,mode: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})

In [24]:
for link in links.findall('link'):
    linkid = link.get('linkId')
    
    if linkid in dict_train.keys():
        
        for attributes in link.findall('attributes'):
            b=False
            attr_train=None
            mode ='train'

            for way_id , ligne in dict_train[linkid].items():
                for lg in ligne:
                    for relation_i, relation_n in lg.items():
                        
                        for attribute in attributes.findall('attribute'):
                            if (attribute.get('name')==mode):
                                b=True
                                attr_train=attribute
                                break
                        
                        ligne_name=relation_n.split(' ')
                        if (b):
                            attr_train.set(mode, attr_train.get(mode)+' '+ 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1])
                        else:
                            ET.SubElement(attributes, 'attribute',{'Type':'Transport','name':mode,mode: 'lingeId:'+relation_i+','+ligne_name[0]+':'+ligne_name[1]})

In [25]:
tree = ET.ElementTree(root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')

```
    <link linkId="24173" length="9.344" node1="1207167" node2="1207173">
      <attributes>
        <attribute Type="Topology" name="nodes" nodeId="1207167 1207168 1207169 1207170 1207171 1207172 1207173" />
        <attribute Type="Topology" name="width" width=" 33.71445229286025 33.6275770990463" />
        <attribute Type="Topology" name="osm_way" way_id="193939965 4039025 13424116 178828670" />
        <attribute Type="Topology" name="osm_link" link_id="81526 81527" />
        <attribute Type="Topology" name="freespeed" freespeed="8.333333333333334 8.333333333333334" />
        <attribute Type="Topology" name="capacity" capacity="3600.0 3600.0" />
        <attribute Type="Topology" name="permlanes" permlanes="3.0 3.0" />
        <attribute Type="Topology" name="oneway" oneway="1 1" />
        <attribute Type="Topology" name="modes" modes="car,pt,bus,car_passenger car,pt,bus,car_passenger" />
        <attribute Type="Topology" name="edge_centrality" edge_centrality="0.020743958138208868 0.020442535203547017" />
        <attribute Type="Transport" name="bus" bus="lingeId:1255723,Bus:29 lingeId:1257181,Bus:56" />
        <attribute Type="Transport" name="subway" subway="lingeId:3328764,Métro:6" />
        <attribute Type="Transport" name="train" train="lingeId:2333438,RER:A1 lingeId:2333440,RER:A1 lingeId:2333442,RER:A3 lingeId:2333445,RER:A3 lingeId:2333447,RER:A5 lingeId:3012957,RER:A5" />
      </attributes>
    </link>

```